Simple least square solve with u.n_s = v_s.n_s

In [9]:
import numpy as np
import scipy.optimize

In [37]:
u0 = np.array([1, 1, 0])
u1 = np.array([0, 1, 1])
u2 = np.array([1, 0, 1])
U = np.concatenate([u0, u1, u2]) # given as such from C++ program

n0 = np.array([1, 0, 0])
n1 = np.array([0, 1, 0])
n2 = np.array([0, 0, 1])
N = np.concatenate([n0, n1, n2]) # given as such from C++ program

V = N # given as such from C++ program

#######################################################

nU = U.size // 3
A = np.zeros([nU, 3 * nU])
B = np.zeros([nU])
for row in range(nU):
    A[row, 3*row : 3*(row+1)] = N[3*row : 3*(row+1)]
    B[row] = np.dot(V[3*row:3*(row+1)], N[3*row:3*(row+1)])

print(A @ U)
print(B)

cons = ({'type': 'eq', 'fun': lambda x:  A @ x - B})

def f(x):
    return np.sum(x**2)

scipy.optimize.minimize(f, U, constraints=cons)

[1. 1. 1.]
[1. 1. 1.]


     fun: 3.0
     jac: array([2., 0., 0., 0., 2., 0., 0., 0., 2.])
 message: 'Optimization terminated successfully'
    nfev: 21
     nit: 2
    njev: 2
  status: 0
 success: True
       x: array([1., 0., 0., 0., 1., 0., 0., 0., 1.])